In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
# Read the data
X = pd.read_csv('/content/train.csv')
X_test_full = pd.read_csv('/content/test.csv')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice             
X.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)


In [4]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
my_model=XGBRegressor(n_estimators=3000, learning_rate=0.05)
my_model.fit(X_train, y_train, early_stopping_rounds = 150, eval_set=[(X_valid, y_valid)], verbose=False)

[18:45:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=3000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [8]:
prediction = my_model.predict(X_valid)
mae=mean_absolute_error(prediction, y_valid)
print(y_valid.head())
print(mae)
print(y_valid.mean())
print(mae/y_valid.mean())


529    200624
491    133000
459    110000
279    192000
655     88000
Name: SalePrice, dtype: int64
16290.316339362158
181370.38356164383
0.08981795163831384


In [6]:
print(prediction[4])

91329.28
